In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os, glob, pickle, time, gc, copy, sys
import warnings
from tqdm import tqdm
import re

tqdm.pandas()
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 100) # 表示できる列数

sys.path.append('../src')
from utils import ri, pickle_load, pickle_save
import nltk

In [2]:
# config
params = {}
### params directory
params['INPUT_DIR'] = "../input/orig"
params['OUTPUT_DIR'] = "../output"

In [3]:
df_train = pd.read_csv(params['INPUT_DIR']+"/train.csv")
df_train.head()

,Id,pub_title,dataset_title,dataset_label,cleaned_label
0,d0fa7568-7d8e-4db9-870f-f9c6f668c17b,The Impact of Dual Enrollment on College Degre...,National Education Longitudinal Study,National Education Longitudinal Study,national education longitudinal study
1,2f26f645-3dec-485d-b68d-f013c9e05e60,Educational Attainment of High School Dropouts...,National Education Longitudinal Study,National Education Longitudinal Study,national education longitudinal study
2,c5d5cd2c-59de-4f29-bbb1-6a88c7b52f29,Differences in Outcomes for Female and Male St...,National Education Longitudinal Study,National Education Longitudinal Study,national education longitudinal study
3,5c9a3bc9-41ba-4574-ad71-e25c1442c8af,Stepping Stone and Option Value in a Model of ...,National Education Longitudinal Study,National Education Longitudinal Study,national education longitudinal study
4,c754dec7-c5a3-4337-9892-c02158475064,"Parental Effort, School Resources, and Student...",National Education Longitudinal Study,National Education Longitudinal Study,national education longitudinal study


In [4]:
df_train_reduced = pickle_load(params['OUTPUT_DIR']+"/df_train_reduced.pkl")
df_train_reduced.head()

,Id,pub_title,dataset_title,dataset_label,cleaned_label,text,clean_text
0,d0fa7568-7d8e-4db9-870f-f9c6f668c17b,The Impact of Dual Enrollment on College Degre...,National Education Longitudinal Study,National Education Longitudinal Study,national education longitudinal study,This study used data from the National Educati...,this study used data from the national educati...
1,2f26f645-3dec-485d-b68d-f013c9e05e60,Educational Attainment of High School Dropouts...,National Education Longitudinal Study,National Education Longitudinal Study,national education longitudinal study,Dropping out of high school is not necessarily...,dropping out of high school is not necessarily...
2,c5d5cd2c-59de-4f29-bbb1-6a88c7b52f29,Differences in Outcomes for Female and Male St...,National Education Longitudinal Study,National Education Longitudinal Study,national education longitudinal study,", stress satisfactory outcomes for all youth,...",stress satisfactory outcomes for all youth inc...
3,5c9a3bc9-41ba-4574-ad71-e25c1442c8af,Stepping Stone and Option Value in a Model of ...,National Education Longitudinal Study,National Education Longitudinal Study,national education longitudinal study,Federal Reserve Bank of Richmond S1. Accountin...,federal reserve bank of richmond s1 accounting...
4,c754dec7-c5a3-4337-9892-c02158475064,"Parental Effort, School Resources, and Student...",National Education Longitudinal Study,National Education Longitudinal Study,national education longitudinal study,This article investigates an important factor ...,this article investigates an important factor ...


# Train label

In [5]:
def jaccard(str1, str2): 
    a = set(str1.lower().split()) 
    b = set(str2.lower().split())
    if len(a)==0 or len(b)==0:
        return 0
    c = a.intersection(b)
    return float(len(c)) / (len(a) + len(b) - len(c))

def clean_text(txt):
    return re.sub('[^A-Za-z0-9]+', ' ', str(txt).lower()).strip()

In [6]:
df_label = df_train['cleaned_label'].value_counts().reset_index()
df_label.columns = ['cleaned_label', 'count']
df_label['-count'] = -df_label['count']
df_label = df_label.sort_values(['-count', 'cleaned_label']).reset_index(drop=True)
df_label['target'] = np.arange(len(df_label))
df_label['cleaned_label'] = df_label['cleaned_label'].apply(clean_text)

df_tmp1 = df_train[['dataset_label', 'cleaned_label', 'dataset_title']]
df_tmp2 = df_train[['dataset_title', 'cleaned_label', 'dataset_title']]
df_tmp3 = df_train[['cleaned_label', 'cleaned_label', 'dataset_title']]
df_tmp1.columns = ['label', 'cleaned_label', 'dataset_title']
df_tmp2.columns = ['label', 'cleaned_label', 'dataset_title']
df_tmp3.columns = ['label', 'cleaned_label', 'dataset_title']
df_label2 = pd.concat([df_tmp1, df_tmp2, df_tmp3])

# df_label2['label'] = df_label2['label'].apply(lambda x: x.lower())
df_label2['cleaned_label_my'] = df_label2['label'].apply(clean_text)
df_label2['cleaned_label'] = df_label2['cleaned_label'].apply(clean_text)
df_label2 = df_label2[df_label2['label'].duplicated()==False].reset_index(drop=True)
df_label2 = pd.merge(df_label2, df_label, on='cleaned_label', how='left')
print(df_label2.shape)
print(np.sum(pd.isna(df_label2['target'])))
# print(df_label2[df_label2['target']==1])
df_label2 = df_label2.sort_values(['-count', 'label']).reset_index(drop=True)
df_label2['target_relabeled'] = df_label2['target']
df_label2['target_relabeled'][df_label2['cleaned_label']!=df_label2['cleaned_label_my']] =\
    np.arange(np.sum(df_label2['cleaned_label']!=df_label2['cleaned_label_my'])) + df_label2['target'].max()+1
print("not match: ", np.sum(df_label2['cleaned_label']!=df_label2['cleaned_label_my']))
print("unique cleaned_label_my: ", len(df_label2['cleaned_label_my'].unique()))
print(df_label2['target'].max())
print(df_label2['target_relabeled'].max())

df_label_train = df_label2.copy()
print(df_label_train.shape)
df_label_train.head()

(263, 7)
0
not match:  3
unique cleaned_label_my:  133
129
132
(263, 8)


,label,cleaned_label,dataset_title,cleaned_label_my,count,-count,target,target_relabeled
0,ADNI,adni,Alzheimer's Disease Neuroimaging Initiative (A...,adni,3673,-3673,0,0
1,adni,adni,Alzheimer's Disease Neuroimaging Initiative (A...,adni,3673,-3673,0,0
2,Alzheimer's Disease Neuroimaging Initiative (A...,alzheimer s disease neuroimaging initiative adni,Alzheimer's Disease Neuroimaging Initiative (A...,alzheimer s disease neuroimaging initiative adni,2400,-2400,1,1
3,alzheimer s disease neuroimaging initiative adni,alzheimer s disease neuroimaging initiative adni,Alzheimer's Disease Neuroimaging Initiative (A...,alzheimer s disease neuroimaging initiative adni,2400,-2400,1,1
4,Trends in International Mathematics and Scienc...,trends in international mathematics and scienc...,Trends in International Mathematics and Scienc...,trends in international mathematics and scienc...,1163,-1163,2,2


# train acronym

In [7]:

acronym_list = [
    ['National Education Longitudinal Study', 'nels'],
#     ['NOAA Tide Gauge', 'nan'],
    ['Sea, Lake, and Overland Surges from Hurricanes', 'slosh'],
    ['Coastal Change Analysis Program', 'c-cap'],
    ['Aging Integrated Database (AGID)', 'agid'],
    ["Alzheimer's Disease Neuroimaging Initiative (ADNI)", 'adni'],
    ['Baltimore Longitudinal Study of Aging (BLSA)', 'blsa'],
    ['Agricultural Resource Management Survey', 'arms'],
    ['Beginning Postsecondary Student', 'bps'],
    ["The National Institute on Aging Genetics of Alzheimer's Disease Data Storage Site (NIAGADS)", 'niagads'],
    ['Common Core of Data', 'ccd'],
#     ['Survey of Industrial Research and Development', 'nan'],
    ['Baccalaureate and Beyond', 'b&b'],
    ['International Best Track Archive for Climate Stewardship', 'IBTrACS'],
    ['National Teacher and Principal Survey', 'ntps'],
    ['Higher Education Research and Development Survey', 'herd'],
    ['Survey of Earned Doctorates', 'sed'],
    ['School Survey on Crime and Safety', 'ssocs'],
    ['World Ocean Database', 'wod'],
    ['Program for the International Assessment of Adult Competencies', 'piaac'],
    ['Early Childhood Longitudinal Study', 'ecls'],
#     ['Survey of Graduate Students and Postdoctorates in Science and Engineering', 'nan'],
    ['Trends in International Mathematics and Science Study', 'timss'],
    ['Education Longitudinal Study', 'els'],
    ['Optimum Interpolation Sea Surface Temperature', 'oisst'],
    ['National Assessment of Education Progress', 'naep'],
    ['High School Longitudinal Study', 'hsls'],
    ['Survey of Doctorate Recipients', 'sdr'],
    ['Rural-Urban Continuum Codes', 'rucc'],
#     ['Survey of Science and Engineering Research Facilities', 'nan'],
#     ['FFRDC Research and Development Survey', 'nan'],
#     ['Survey of State Government Research and Development', 'nan'],
    ['Advanced National Seismic System (ANSS) Comprehensive Catalog (ComCat)', 'comcat'],
#     ['Census of Agriculture', 'nan'],
    ['North American Breeding Bird Survey (BBS)', 'bbs'],
    ['COVID-19 Open Research Dataset (CORD-19)', 'cord-19'],
    ['Complexity Science Hub COVID-19 Control Strategies List (CCCSL)', 'cccsl'],
#     ['Our World in Data COVID-19 dataset', 'nan'],
    ['COVID-19 Precision Medicine Analytics Platform Registry (JH-CROWN)', 'jh-crown'],
    ['Characterizing Health Associated Risks, and Your Baseline Disease In SARS-COV-2 (CHARYBDIS)', 'charybdis'],
#     ['COVID-19 Deaths data', 'nan'],
#     ['SARS-CoV-2 genome sequence', 'nan'],
#     ['COVID-19 Image Data Collection', 'nan'],
    ['RSNA International COVID-19 Open Radiology Database (RICORD)', 'ricord'],
#     ['CAS COVID-19 antiviral candidate compounds dataset', 'nan'],
]
df_label_train_acronym = pd.DataFrame(acronym_list, columns=['dataset_title', 'acronym'])
df_label_train_acronym['acronym_clean'] = df_label_train_acronym['acronym'].apply(clean_text)
df_label_train_acronym.head()

,dataset_title,acronym,acronym_clean
0,National Education Longitudinal Study,nels,nels
1,"Sea, Lake, and Overland Surges from Hurricanes",slosh,slosh
2,Coastal Change Analysis Program,c-cap,c cap
3,Aging Integrated Database (AGID),agid,agid
4,Alzheimer's Disease Neuroimaging Initiative (A...,adni,adni


In [8]:
df_label_train2 = pd.merge(df_label_train, df_label_train_acronym, on='dataset_title', how='left')
df_label_train2['cleaned_label'] = df_label_train2['cleaned_label_my']
df_label_train2 = df_label_train2[['label', 'cleaned_label', 'acronym', 'acronym_clean']]
df_label_train2['ver'] = 0
print(df_label_train2.shape)
df_label_train2.head()

(263, 5)


,label,cleaned_label,acronym,acronym_clean,ver
0,ADNI,adni,adni,adni,0
1,adni,adni,adni,adni,0
2,Alzheimer's Disease Neuroimaging Initiative (A...,alzheimer s disease neuroimaging initiative adni,adni,adni,0
3,alzheimer s disease neuroimaging initiative adni,alzheimer s disease neuroimaging initiative adni,adni,adni,0
4,Trends in International Mathematics and Scienc...,trends in international mathematics and scienc...,timss,timss,0


# Detect acronym ver1

In [9]:
def detect_keywords(text, keywords):
    for keyword in keywords:
        if keyword in text:
            return True
    return False

def det_acronym_ver1(text, keywords, TH_LEN_CHAR = 3):
    ans = []
    # text = re.sub("-", " ", text)
    words = text.split()
    for i, word in enumerate(words):
        if word[0]!='(' or word[-1]!=')': continue # (XXX)の形でなければスルー
        acronym_cand = word[1:-1] # ()の除去
        if acronym_cand.lower()==acronym_cand: continue # 大文字が一つもないならスルー
        len_acronym_cand = len(acronym_cand)
        if len_acronym_cand<TH_LEN_CHAR: continue # 3文字以下ならスルー
        acronym_cand_lower = acronym_cand.lower() # 小文字変換
        acronym_cand_reverse = acronym_cand_lower[::-1] #並びを反転
        
        words_cand = words[np.clip(i-len_acronym_cand*3, 0, i):i] # (XXX)の前の数単語を抽出
        words_cand = ' '.join(words_cand).strip()
        words_cand = re.sub('[^A-Za-z0-9]+', ' ', words_cand).strip() # 記号除去
        words_cand = words_cand.split(" ")
        words_cand_reverse = words_cand[::-1]
        
        idx = 0 # acronym_candの文字index
        for j, word in enumerate(words_cand_reverse):
            if idx==len_acronym_cand:
                break
            if len(word)==0:
                continue
            if word[0].lower()==acronym_cand_reverse[idx]:
                if idx==len_acronym_cand-1: # 1文字目を検出
                    idx_start = j # dataset名の1単語目のindex
                idx += 1
        if idx==len_acronym_cand:
            words_reverse = words_cand_reverse[:idx_start+1]
            dataset = ' '.join(words_reverse[::-1]).strip().lower()
            if detect_keywords(dataset, keywords):
                acronym = acronym_cand
#                 print("Acronym: {}".format(acronym))
#                 print("Dataset: {}".format(dataset))
                ans.append([acronym_cand, dataset])
        return ans
    
keywords = [
    'study',
    'studies',
    'data',
    'survey',
    'panel',
    'census',
    'cohort',
    'longitudinal',
    'registry',
]    
df_train_reduced['det_acronym'] = df_train_reduced['text'].progress_apply(lambda x: det_acronym_ver1(x, keywords))
df_train_reduced.head()

100%|██████████| 14316/14316 [00:08<00:00, 1675.23it/s]


,Id,pub_title,dataset_title,dataset_label,cleaned_label,text,clean_text,det_acronym
0,d0fa7568-7d8e-4db9-870f-f9c6f668c17b,The Impact of Dual Enrollment on College Degre...,National Education Longitudinal Study,National Education Longitudinal Study,national education longitudinal study,This study used data from the National Educati...,this study used data from the national educati...,[]
1,2f26f645-3dec-485d-b68d-f013c9e05e60,Educational Attainment of High School Dropouts...,National Education Longitudinal Study,National Education Longitudinal Study,national education longitudinal study,Dropping out of high school is not necessarily...,dropping out of high school is not necessarily...,[]
2,c5d5cd2c-59de-4f29-bbb1-6a88c7b52f29,Differences in Outcomes for Female and Male St...,National Education Longitudinal Study,National Education Longitudinal Study,national education longitudinal study,", stress satisfactory outcomes for all youth,...",stress satisfactory outcomes for all youth inc...,[]
3,5c9a3bc9-41ba-4574-ad71-e25c1442c8af,Stepping Stone and Option Value in a Model of ...,National Education Longitudinal Study,National Education Longitudinal Study,national education longitudinal study,Federal Reserve Bank of Richmond S1. Accountin...,federal reserve bank of richmond s1 accounting...,[]
4,c754dec7-c5a3-4337-9892-c02158475064,"Parental Effort, School Resources, and Student...",National Education Longitudinal Study,National Education Longitudinal Study,national education longitudinal study,This article investigates an important factor ...,this article investigates an important factor ...,"[[NELS, national education longitudinal study]]"


In [10]:
tmp = []
# ri(df_traintest)なしだとここでエラー
for i in range(len(df_train_reduced)):
    if df_train_reduced['det_acronym'][i] is not None:
        tmp += df_train_reduced['det_acronym'][i]
df_train_acronym = pd.DataFrame(tmp)
df_train_acronym.columns = ['acronym', 'base']
df_train_acronym['base_acronym'] = df_train_acronym['base']+"|"+df_train_acronym['acronym']
df_train_acronym = df_train_acronym[df_train_acronym['base_acronym'].duplicated()==False]
df_train_acronym['train'] = True
print(df_train_acronym.shape)
# acronym unique =256
# BA unique 325
df_train_acronym.head(30)

(325, 4)


,acronym,base,base_acronym,train
0,NELS,national education longitudinal study,national education longitudinal study|NELS,True
1,NLS,national longitudinal study,national longitudinal study|NLS,True
2,BPS,beginning postsecondary student longitudinal s...,beginning postsecondary student longitudinal s...,True
3,NELS,national educational longitudinal survey of 1988,national educational longitudinal survey of 19...,True
4,HSB,high school and beyond longitudinal dataset,high school and beyond longitudinal dataset|HSB,True
5,NELS,national education longitudinal study of 1988,national education longitudinal study of 1988|...,True
8,NHES,national household education survey,national household education survey|NHES,True
10,NLSY,national longitudinal survey of youth,national longitudinal survey of youth|NLSY,True
11,ATUS,american time use survey,american time use survey|ATUS,True
12,TIMSS,trends in international mathematics and scienc...,trends in international mathematics and scienc...,True


In [11]:
df_label_acronym_ver1_all = df_train_acronym.copy()
print(df_label_acronym_ver1_all.shape)
df_label_acronym_ver1_all.head()

(325, 4)


,acronym,base,base_acronym,train
0,NELS,national education longitudinal study,national education longitudinal study|NELS,True
1,NLS,national longitudinal study,national longitudinal study|NLS,True
2,BPS,beginning postsecondary student longitudinal s...,beginning postsecondary student longitudinal s...,True
3,NELS,national educational longitudinal survey of 1988,national educational longitudinal survey of 19...,True
4,HSB,high school and beyond longitudinal dataset,high school and beyond longitudinal dataset|HSB,True


In [12]:
def get_match(x, ref_labels, threshold=0.5):
    for label in ref_labels:
        if jaccard(x, label)>=threshold:
            return label
    return 'no_match'

df_label_acronym_ver1_all['match'] = df_label_acronym_ver1_all['base'].apply(get_match, ref_labels=df_label2['cleaned_label_my'])
print((df_label_acronym_ver1_all['match']=='no_match').sum())
print(df_label_acronym_ver1_all.shape)
df_label_acronym_ver1_all.head()

270
(325, 5)


,acronym,base,base_acronym,train,match
0,NELS,national education longitudinal study,national education longitudinal study|NELS,True,education longitudinal study
1,NLS,national longitudinal study,national longitudinal study|NLS,True,education longitudinal study
2,BPS,beginning postsecondary student longitudinal s...,beginning postsecondary student longitudinal s...,True,beginning postsecondary students longitudinal ...
3,NELS,national educational longitudinal survey of 1988,national educational longitudinal survey of 19...,True,no_match
4,HSB,high school and beyond longitudinal dataset,high school and beyond longitudinal dataset|HSB,True,no_match


In [13]:
df_label_acronym_ver1_all[(df_label_acronym_ver1_all['match']!='no_match')&(df_label_acronym_ver1_all['match'].duplicated())].head(30)

,acronym,base,base_acronym,train,match
1,NLS,national longitudinal study,national longitudinal study|NLS,True,education longitudinal study
5,NELS,national education longitudinal study of 1988,national education longitudinal study of 1988|...,True,education longitudinal study
19,WLS,wisconsin longitudinal study,wisconsin longitudinal study|WLS,True,education longitudinal study
21,NELS,national education longitudinal study 1988 stu...,national education longitudinal study 1988 stu...,True,education longitudinal study
160,ARMS,and resource management survey,and resource management survey|ARMS,True,agricultural resource management survey
190,BPS,beginning postsecondary students longitudinal ...,beginning postsecondary students longitudinal ...,True,beginning postsecondary students
191,NLTS2,national longitudinal transition study 2,national longitudinal transition study 2|NLTS2,True,national education longitudinal study
197,TIMSS,third international mathematics and science study,third international mathematics and science st...,True,trends in international mathematics and scienc...
220,TLSA,taiwan longitudinal study of aging,taiwan longitudinal study of aging|TLSA,True,baltimore longitudinal study of aging
254,TIMSS,trends in mathematics and science study,trends in mathematics and science study|TIMSS,True,trends in international mathematics and scienc...


In [14]:
def delete_keywords(x, keywords):
    keywords += ['of', 'the', 'national', 'education']
    x_split = x.split()
    x_new = []
    for word in x_split:
        if word not in keywords:
            x_new.append(word)
    x_new = ' '.join(x_new)
    return x_new
    
df_label2['label-keywords'] = df_label2['cleaned_label_my'].apply(lambda x: delete_keywords(x, keywords))
df_label2.head()

,label,cleaned_label,dataset_title,cleaned_label_my,count,-count,target,target_relabeled,label-keywords
0,ADNI,adni,Alzheimer's Disease Neuroimaging Initiative (A...,adni,3673,-3673,0,0,adni
1,adni,adni,Alzheimer's Disease Neuroimaging Initiative (A...,adni,3673,-3673,0,0,adni
2,Alzheimer's Disease Neuroimaging Initiative (A...,alzheimer s disease neuroimaging initiative adni,Alzheimer's Disease Neuroimaging Initiative (A...,alzheimer s disease neuroimaging initiative adni,2400,-2400,1,1,alzheimer s disease neuroimaging initiative adni
3,alzheimer s disease neuroimaging initiative adni,alzheimer s disease neuroimaging initiative adni,Alzheimer's Disease Neuroimaging Initiative (A...,alzheimer s disease neuroimaging initiative adni,2400,-2400,1,1,alzheimer s disease neuroimaging initiative adni
4,Trends in International Mathematics and Scienc...,trends in international mathematics and scienc...,Trends in International Mathematics and Scienc...,trends in international mathematics and scienc...,1163,-1163,2,2,trends in international mathematics and science


In [15]:
df_label_acronym_ver1_all['label-keywords'] = df_label_acronym_ver1_all['base'].apply(lambda x: delete_keywords(x, keywords))
df_label_acronym_ver1_all['len_label-keywords'] = df_label_acronym_ver1_all['label-keywords'].apply(lambda x: len(x.split()))
df_label_acronym_ver1_all.head()

,acronym,base,base_acronym,train,match,label-keywords,len_label-keywords
0,NELS,national education longitudinal study,national education longitudinal study|NELS,True,education longitudinal study,,0
1,NLS,national longitudinal study,national longitudinal study|NLS,True,education longitudinal study,,0
2,BPS,beginning postsecondary student longitudinal s...,beginning postsecondary student longitudinal s...,True,beginning postsecondary students longitudinal ...,beginning postsecondary student,3
3,NELS,national educational longitudinal survey of 1988,national educational longitudinal survey of 19...,True,no_match,educational 1988,2
4,HSB,high school and beyond longitudinal dataset,high school and beyond longitudinal dataset|HSB,True,no_match,high school and beyond dataset,5


In [16]:
df_label_acronym_ver1_all['match_train-keywords'] = df_label_acronym_ver1_all['label-keywords'].apply(lambda x:
    get_match(x, ref_labels=df_label2['label-keywords'], threshold=0.75))
df_label_acronym_ver1_all.head()

,acronym,base,base_acronym,train,match,label-keywords,len_label-keywords,match_train-keywords
0,NELS,national education longitudinal study,national education longitudinal study|NELS,True,education longitudinal study,,0,no_match
1,NLS,national longitudinal study,national longitudinal study|NLS,True,education longitudinal study,,0,no_match
2,BPS,beginning postsecondary student longitudinal s...,beginning postsecondary student longitudinal s...,True,beginning postsecondary students longitudinal ...,beginning postsecondary student,3,beginning postsecondary student
3,NELS,national educational longitudinal survey of 1988,national educational longitudinal survey of 19...,True,no_match,educational 1988,2,no_match
4,HSB,high school and beyond longitudinal dataset,high school and beyond longitudinal dataset|HSB,True,no_match,high school and beyond dataset,5,no_match


In [17]:
df_label_acronym_ver1_all[
    (df_label_acronym_ver1_all['match_train-keywords']!='no_match')
    &(df_label_acronym_ver1_all['match_train-keywords'].duplicated())].head(30)

,acronym,base,base_acronym,train,match,label-keywords,len_label-keywords,match_train-keywords
190,BPS,beginning postsecondary students longitudinal ...,beginning postsecondary students longitudinal ...,True,beginning postsecondary students,beginning postsecondary students,3,beginning postsecondary students
254,TIMSS,trends in mathematics and science study,trends in mathematics and science study|TIMSS,True,trends in international mathematics and scienc...,trends in mathematics and science,5,trends in international mathematics and science
540,TIMSS,trends in international mathematics and scienc...,trends in international mathematics and scienc...,True,trends in international mathematics and scienc...,trends in international mathematics and scienc...,7,trends in international mathematics and science
636,CCD,common core data,common core data|CCD,True,common core of data,common core,2,common core
689,BPS,beginning postsecondary students longitudinal ...,beginning postsecondary students longitudinal ...,True,beginning postsecondary students,beginning postsecondary students,3,beginning postsecondary students
768,ECLS,early childhood longitudinal studies,early childhood longitudinal studies|ECLS,True,early childhood longitudinal study,early childhood,2,early childhood
1008,ARMS,agricultural resource management survey data,agricultural resource management survey data|ARMS,True,agricultural resource management survey,agricultural resource management,3,agricultural resource management
1033,ARMS,agricultural resource and management survey,agricultural resource and management survey|ARMS,True,agricultural resource management survey,agricultural resource and management,4,agricultural resource management
1035,ARMS,agricultural resource management surveys,agricultural resource management surveys|ARMS,True,agricultural resource management survey,agricultural resource management surveys,4,agricultural resource management
1044,BPS,beginning postsecondary students longitudinal ...,beginning postsecondary students longitudinal ...,True,beginning postsecondary students,beginning postsecondary students 2,4,beginning postsecondary students


In [18]:
idx_tmp = df_label_acronym_ver1_all['match_train-keywords']=='no_match'
print(df_label_acronym_ver1_all[idx_tmp].shape)
# high school effectiveness studyのみ誤削除
df_label_acronym_ver1_all[idx_tmp==False]

(298, 8)


,acronym,base,base_acronym,train,match,label-keywords,len_label-keywords,match_train-keywords
2,BPS,beginning postsecondary student longitudinal s...,beginning postsecondary student longitudinal s...,True,beginning postsecondary students longitudinal ...,beginning postsecondary student,3,beginning postsecondary student
12,TIMSS,trends in international mathematics and scienc...,trends in international mathematics and scienc...,True,trends in international mathematics and scienc...,trends in international mathematics and science,6,trends in international mathematics and science
22,BPS,beginning postsecondary students longitudinal ...,beginning postsecondary students longitudinal ...,True,beginning postsecondary students,beginning postsecondary students,3,beginning postsecondary students
64,BLSA,baltimore longitudinal study of aging,baltimore longitudinal study of aging|BLSA,True,baltimore longitudinal study of aging,baltimore aging,2,baltimore aging
131,CCD,common core of data,common core of data|CCD,True,common core of data,common core,2,common core
148,ARMS,agricultural resource management survey,agricultural resource management survey|ARMS,True,agricultural resource management survey,agricultural resource management,3,agricultural resource management
187,SSOCS,school survey on crime and safety,school survey on crime and safety|SSOCS,True,school survey on crime and safety,school on crime and safety,5,school on crime and safety
190,BPS,beginning postsecondary students longitudinal ...,beginning postsecondary students longitudinal ...,True,beginning postsecondary students,beginning postsecondary students,3,beginning postsecondary students
208,SED,survey of earned doctorates,survey of earned doctorates|SED,True,survey of earned doctorates,earned doctorates,2,earned doctorates
233,ECLS,early childhood longitudinal study,early childhood longitudinal study|ECLS,True,early childhood longitudinal study,early childhood,2,early childhood


In [19]:
idx_tmp = df_label_acronym_ver1_all['match']=='no_match'
print(df_label_acronym_ver1_all[idx_tmp].shape)
df_label_acronym_ver1_all[idx_tmp==False]

(270, 8)


,acronym,base,base_acronym,train,match,label-keywords,len_label-keywords,match_train-keywords
0,NELS,national education longitudinal study,national education longitudinal study|NELS,True,education longitudinal study,,0,no_match
1,NLS,national longitudinal study,national longitudinal study|NLS,True,education longitudinal study,,0,no_match
2,BPS,beginning postsecondary student longitudinal s...,beginning postsecondary student longitudinal s...,True,beginning postsecondary students longitudinal ...,beginning postsecondary student,3,beginning postsecondary student
5,NELS,national education longitudinal study of 1988,national education longitudinal study of 1988|...,True,education longitudinal study,1988,1,no_match
12,TIMSS,trends in international mathematics and scienc...,trends in international mathematics and scienc...,True,trends in international mathematics and scienc...,trends in international mathematics and science,6,trends in international mathematics and science
19,WLS,wisconsin longitudinal study,wisconsin longitudinal study|WLS,True,education longitudinal study,wisconsin,1,no_match
21,NELS,national education longitudinal study 1988 stu...,national education longitudinal study 1988 stu...,True,education longitudinal study,1988 1994,2,no_match
22,BPS,beginning postsecondary students longitudinal ...,beginning postsecondary students longitudinal ...,True,beginning postsecondary students,beginning postsecondary students,3,beginning postsecondary students
34,NELS,national education longitudinal survey,national education longitudinal survey|NELS,True,national education longitudinal study,,0,no_match
37,HSES,high school effectiveness study,high school effectiveness study|HSES,True,high school longitudinal study,high school effectiveness,3,no_match


In [20]:
df_label_acronym_ver1_all['acronym_clean'] = df_label_acronym_ver1_all['acronym'].apply(clean_text)
df_label_acronym_ver1_all['match_train_acronym'] = df_label_acronym_ver1_all['acronym_clean'].apply(lambda x:
    x in df_label_train_acronym['acronym_clean'].tolist()
                                                                                                   )
df_label_acronym_ver1_all[df_label_acronym_ver1_all['match_train_acronym']]

,acronym,base,base_acronym,train,match,label-keywords,len_label-keywords,match_train-keywords,acronym_clean,match_train_acronym
0,NELS,national education longitudinal study,national education longitudinal study|NELS,True,education longitudinal study,,0,no_match,nels,True
2,BPS,beginning postsecondary student longitudinal s...,beginning postsecondary student longitudinal s...,True,beginning postsecondary students longitudinal ...,beginning postsecondary student,3,beginning postsecondary student,bps,True
3,NELS,national educational longitudinal survey of 1988,national educational longitudinal survey of 19...,True,no_match,educational 1988,2,no_match,nels,True
5,NELS,national education longitudinal study of 1988,national education longitudinal study of 1988|...,True,education longitudinal study,1988,1,no_match,nels,True
12,TIMSS,trends in international mathematics and scienc...,trends in international mathematics and scienc...,True,trends in international mathematics and scienc...,trends in international mathematics and science,6,trends in international mathematics and science,timss,True
21,NELS,national education longitudinal study 1988 stu...,national education longitudinal study 1988 stu...,True,education longitudinal study,1988 1994,2,no_match,nels,True
22,BPS,beginning postsecondary students longitudinal ...,beginning postsecondary students longitudinal ...,True,beginning postsecondary students,beginning postsecondary students,3,beginning postsecondary students,bps,True
23,ECLS,early childhood longitudinal study kindergarte...,early childhood longitudinal study kindergarte...,True,no_match,early childhood kindergarten class 1998 99,6,no_match,ecls,True
34,NELS,national education longitudinal survey,national education longitudinal survey|NELS,True,national education longitudinal study,,0,no_match,nels,True
64,BLSA,baltimore longitudinal study of aging,baltimore longitudinal study of aging|BLSA,True,baltimore longitudinal study of aging,baltimore aging,2,baltimore aging,blsa,True


In [21]:
# NLTS2, ECLSKが問題か
print(df_label_acronym_ver1_all.shape)
df_label_acronym_ver1_all.to_csv(params['OUTPUT_DIR']+"/df_label_acronym_ver1_all.csv", index=None)

(325, 10)


In [22]:
idx_tmp = df_label_acronym_ver1_all['len_label-keywords']>0
# idx_tmp = idx_tmp & (df_label_acronym_ver1_all['match_train_acronym']==False)
# idx_tmp = idx_tmp & (df_label_acronym_ver1_all['match_train-keywords']=='no_match')
df_label_acronym_ver1_filtered = df_label_acronym_ver1_all[idx_tmp].reset_index(drop=True)
print(df_label_acronym_ver1_filtered.shape)
df_label_acronym_ver1_filtered['label'] = df_label_acronym_ver1_filtered['base'].apply(clean_text)
df_label_acronym_ver1_filtered['cleaned_label'] = df_label_acronym_ver1_filtered['label']
df_label_acronym_ver1_filtered = df_label_acronym_ver1_filtered[['label', 'cleaned_label', 'acronym', 'acronym_clean']]
df_label_acronym_ver1_filtered['ver'] = 1
df_label_acronym_ver1_filtered.head()

(319, 10)


,label,cleaned_label,acronym,acronym_clean,ver
0,beginning postsecondary student longitudinal s...,beginning postsecondary student longitudinal s...,BPS,bps,1
1,national educational longitudinal survey of 1988,national educational longitudinal survey of 1988,NELS,nels,1
2,high school and beyond longitudinal dataset,high school and beyond longitudinal dataset,HSB,hsb,1
3,national education longitudinal study of 1988,national education longitudinal study of 1988,NELS,nels,1
4,national household education survey,national household education survey,NHES,nhes,1


# Detect acronym ver2

In [23]:
len_words = []
VERBOSE = 100
starttime = time.time()
for i in range(len(df_train_reduced)):
    if (i+1)%VERBOSE==0:
        print("{}/{}, sec: {:.1f}".format(i+1, len(df_train_reduced), time.time()-starttime))
    tmp = len(df_train_reduced['text'][i].split())
    len_words.append(tmp)

100/14316, sec: 0.0
200/14316, sec: 0.1
300/14316, sec: 0.2
400/14316, sec: 0.2
500/14316, sec: 0.2
600/14316, sec: 0.3
700/14316, sec: 0.3
800/14316, sec: 0.3
900/14316, sec: 0.4
1000/14316, sec: 0.4
1100/14316, sec: 0.5
1200/14316, sec: 0.5
1300/14316, sec: 0.5
1400/14316, sec: 0.6
1500/14316, sec: 0.6
1600/14316, sec: 0.7
1700/14316, sec: 0.7
1800/14316, sec: 0.7
1900/14316, sec: 0.8
2000/14316, sec: 0.8
2100/14316, sec: 0.8
2200/14316, sec: 0.9
2300/14316, sec: 0.9
2400/14316, sec: 0.9
2500/14316, sec: 1.0
2600/14316, sec: 1.0
2700/14316, sec: 1.1
2800/14316, sec: 1.1
2900/14316, sec: 1.2
3000/14316, sec: 1.2
3100/14316, sec: 1.2
3200/14316, sec: 1.3
3300/14316, sec: 1.3
3400/14316, sec: 1.3
3500/14316, sec: 1.4
3600/14316, sec: 1.4
3700/14316, sec: 1.4
3800/14316, sec: 1.5
3900/14316, sec: 1.5
4000/14316, sec: 1.5
4100/14316, sec: 1.6
4200/14316, sec: 1.6
4300/14316, sec: 1.7
4400/14316, sec: 1.7
4500/14316, sec: 1.7
4600/14316, sec: 1.7
4700/14316, sec: 1.8
4800/14316, sec: 1.8
4

In [24]:
df_tmp = pd.DataFrame(len_words, columns=['len'])
print(df_tmp['len'].max())
df_tmp[df_tmp['len']>100000]

1651596


,len
9209,337870
9340,384484
9352,437497
9412,336798
9599,133375
9697,328235
9746,216203
9785,124202
10392,165110
10415,125132


In [25]:
TH_CHAR_LEN = 3
VERBOSE = 100
starttime = time.time()

def detect_acronym2(inputs):
    text_i, i = inputs
    if (i+1)%VERBOSE==0:
        print("{}/{}, sec: {:.1f}".format(i+1, len(df_traintest), time.time()-starttime))
    words_i = text_i.split()
    acronyms = []
    for j, word in enumerate(words_i):
        if word.lower()!=word and word.upper()==word and len(word)>=TH_CHAR_LEN:
            word_tmp = clean_text(word)
            if len(word_tmp.split())>1:continue
            if len(word_tmp)==1:continue
            if len(word_tmp)<TH_CHAR_LEN:continue
            acronyms.append(word.strip("(),.;:'").strip('"'))
    #         print(" ".join(words_i[j-10:j+10]))
    acronyms = np.unique(acronyms)
    acronym_bases = []
    for acronym in acronyms:
    #     print(acronym)
        hit = 0
        detected = -1
        for j, word in enumerate(words_i):
            if len(word)==0: continue
    #         print(word)a
            if hit==0 and word[0]==acronym[hit]:
                hit += 1
                detected = j
            elif word[0]==acronym[hit] and j-detected<3*hit:
                hit += 1
            elif j-detected>=3*hit:
                hit = 0
                detected = -1
            if hit==len(acronym):
    #             print(acronym, ' '.join(words_i[detected:j+1]))
                acronym_bases.append((acronym, ' '.join(words_i[detected:j+1])))
                break
    df_acronym_i = pd.DataFrame(acronym_bases, columns=['acronym', 'base'])
    df_acronym_i['text_index'] = i
    df_acronym_i['keyword'] = df_acronym_i['base'].apply(lambda x: detect_keywords(x.lower(), keywords))
    df_acronym_i.to_csv(params['OUTPUT_DIR']+"/df_i_{}.csv".format(i), index=None)
    return 1
    
    

In [26]:
df_label_acronym_ver2 = pd.read_csv(params['OUTPUT_DIR']+"/df_acronym3_210615.csv")
print(df_label_acronym_ver2.shape)
df_label_acronym_ver2.head()

(1172, 6)


,acronym,base,text_index,keyword,base_acronym,count_BA
0,ARMS,Agricultural Resource Management Survey,972,True,Agricultural Resource Management Survey|ARMS,423
1,BLSA,Baltimore Longitudinal Study of Aging,518,True,Baltimore Longitudinal Study of Aging|BLSA,405
2,NELS,National Education Longitudinal Study,0,True,National Education Longitudinal Study|NELS,258
3,CCD,Common Core of Data,84,True,Common Core of Data|CCD,202
4,IPEDS,Integrated Postsecondary Education Data System,39,True,Integrated Postsecondary Education Data System...,181


In [27]:
df_label_acronym_ver2['ver2'] = True
df_label_acronym_ver2['cleaned_label'] = df_label_acronym_ver2['base'].apply(lambda x: clean_text(x).lower().strip()) 
df_label_acronym_ver2['dataset'] = df_label_acronym_ver2['cleaned_label'].apply(get_match, ref_labels=df_label_acronym_ver1_all['base'])
df_label_acronym_ver2.head()

,acronym,base,text_index,keyword,base_acronym,count_BA,ver2,cleaned_label,dataset
0,ARMS,Agricultural Resource Management Survey,972,True,Agricultural Resource Management Survey|ARMS,423,True,agricultural resource management survey,agricultural resource management survey
1,BLSA,Baltimore Longitudinal Study of Aging,518,True,Baltimore Longitudinal Study of Aging|BLSA,405,True,baltimore longitudinal study of aging,baltimore longitudinal study of aging
2,NELS,National Education Longitudinal Study,0,True,National Education Longitudinal Study|NELS,258,True,national education longitudinal study,national education longitudinal study
3,CCD,Common Core of Data,84,True,Common Core of Data|CCD,202,True,common core of data,common core of data
4,IPEDS,Integrated Postsecondary Education Data System,39,True,Integrated Postsecondary Education Data System...,181,True,integrated postsecondary education data system,integrated postsecondary education data system


In [28]:
df_label_acronym_ver2['label-keywords'] = df_label_acronym_ver2['base'].apply(lambda x: delete_keywords(x.lower(), keywords))
df_label_acronym_ver2['len_label-keywords'] = df_label_acronym_ver2['label-keywords'].apply(lambda x: len(x.split()))
df_label_acronym_ver2.head()

,acronym,base,text_index,keyword,base_acronym,count_BA,ver2,cleaned_label,dataset,label-keywords,len_label-keywords
0,ARMS,Agricultural Resource Management Survey,972,True,Agricultural Resource Management Survey|ARMS,423,True,agricultural resource management survey,agricultural resource management survey,agricultural resource management,3
1,BLSA,Baltimore Longitudinal Study of Aging,518,True,Baltimore Longitudinal Study of Aging|BLSA,405,True,baltimore longitudinal study of aging,baltimore longitudinal study of aging,baltimore aging,2
2,NELS,National Education Longitudinal Study,0,True,National Education Longitudinal Study|NELS,258,True,national education longitudinal study,national education longitudinal study,,0
3,CCD,Common Core of Data,84,True,Common Core of Data|CCD,202,True,common core of data,common core of data,common core,2
4,IPEDS,Integrated Postsecondary Education Data System,39,True,Integrated Postsecondary Education Data System...,181,True,integrated postsecondary education data system,integrated postsecondary education data system,integrated postsecondary system,3


In [29]:
df_label_acronym_ver2['match_train-keywords'] = df_label_acronym_ver2['label-keywords'].apply(lambda x:
    get_match(x, ref_labels=df_label2['label-keywords'], threshold=0.75))
df_label_acronym_ver2.head()

,acronym,base,text_index,keyword,base_acronym,count_BA,ver2,cleaned_label,dataset,label-keywords,len_label-keywords,match_train-keywords
0,ARMS,Agricultural Resource Management Survey,972,True,Agricultural Resource Management Survey|ARMS,423,True,agricultural resource management survey,agricultural resource management survey,agricultural resource management,3,agricultural resource management
1,BLSA,Baltimore Longitudinal Study of Aging,518,True,Baltimore Longitudinal Study of Aging|BLSA,405,True,baltimore longitudinal study of aging,baltimore longitudinal study of aging,baltimore aging,2,baltimore aging
2,NELS,National Education Longitudinal Study,0,True,National Education Longitudinal Study|NELS,258,True,national education longitudinal study,national education longitudinal study,,0,no_match
3,CCD,Common Core of Data,84,True,Common Core of Data|CCD,202,True,common core of data,common core of data,common core,2,common core
4,IPEDS,Integrated Postsecondary Education Data System,39,True,Integrated Postsecondary Education Data System...,181,True,integrated postsecondary education data system,integrated postsecondary education data system,integrated postsecondary system,3,no_match


In [30]:
df_label_acronym_ver2['match_ver1-keywords'] = df_label_acronym_ver2['label-keywords'].apply(lambda x:
    get_match(x, ref_labels=df_label_acronym_ver1_all['label-keywords'], threshold=0.75))
df_label_acronym_ver2.head()

,acronym,base,text_index,keyword,base_acronym,count_BA,ver2,cleaned_label,dataset,label-keywords,len_label-keywords,match_train-keywords,match_ver1-keywords
0,ARMS,Agricultural Resource Management Survey,972,True,Agricultural Resource Management Survey|ARMS,423,True,agricultural resource management survey,agricultural resource management survey,agricultural resource management,3,agricultural resource management,agricultural resource management
1,BLSA,Baltimore Longitudinal Study of Aging,518,True,Baltimore Longitudinal Study of Aging|BLSA,405,True,baltimore longitudinal study of aging,baltimore longitudinal study of aging,baltimore aging,2,baltimore aging,baltimore aging
2,NELS,National Education Longitudinal Study,0,True,National Education Longitudinal Study|NELS,258,True,national education longitudinal study,national education longitudinal study,,0,no_match,no_match
3,CCD,Common Core of Data,84,True,Common Core of Data|CCD,202,True,common core of data,common core of data,common core,2,common core,common core
4,IPEDS,Integrated Postsecondary Education Data System,39,True,Integrated Postsecondary Education Data System...,181,True,integrated postsecondary education data system,integrated postsecondary education data system,integrated postsecondary system,3,no_match,integrated postsecondary system


In [31]:
df_label_acronym_ver2['acronym_clean'] = df_label_acronym_ver2['acronym'].apply(clean_text)
df_label_acronym_ver2['match_train_acronym'] = df_label_acronym_ver2['acronym_clean'].apply(lambda x:
    x in df_label_train_acronym['acronym_clean'].tolist()
)
df_label_acronym_ver2['match_acronym_ver1'] = df_label_acronym_ver2['acronym_clean'].apply(lambda x:
    x in df_label_acronym_ver1_all['acronym_clean'].tolist()
)
df_label_acronym_ver2[df_label_acronym_ver2['match_train_acronym']]

,acronym,base,text_index,keyword,base_acronym,count_BA,ver2,cleaned_label,dataset,label-keywords,len_label-keywords,match_train-keywords,match_ver1-keywords,acronym_clean,match_train_acronym,match_acronym_ver1
0,ARMS,Agricultural Resource Management Survey,972,True,Agricultural Resource Management Survey|ARMS,423,True,agricultural resource management survey,agricultural resource management survey,agricultural resource management,3,agricultural resource management,agricultural resource management,arms,True,True
1,BLSA,Baltimore Longitudinal Study of Aging,518,True,Baltimore Longitudinal Study of Aging|BLSA,405,True,baltimore longitudinal study of aging,baltimore longitudinal study of aging,baltimore aging,2,baltimore aging,baltimore aging,blsa,True,True
2,NELS,National Education Longitudinal Study,0,True,National Education Longitudinal Study|NELS,258,True,national education longitudinal study,national education longitudinal study,,0,no_match,no_match,nels,True,True
3,CCD,Common Core of Data,84,True,Common Core of Data|CCD,202,True,common core of data,common core of data,common core,2,common core,common core,ccd,True,True
5,SDR,Survey of Doctorate Recipients,224,True,Survey of Doctorate Recipients|SDR,160,True,survey of doctorate recipients,survey of doctorate recipients,doctorate recipients,2,doctorate recipients,doctorate recipients,sdr,True,True
6,SED,Survey of Earned Doctorates,1424,True,Survey of Earned Doctorates|SED,153,True,survey of earned doctorates,survey of earned doctorates,earned doctorates,2,earned doctorates,earned doctorates,sed,True,True
19,HSLS,High School Longitudinal Study,2663,True,High School Longitudinal Study|HSLS,35,True,high school longitudinal study,high school effectiveness study,high school,2,high school,no_match,hsls,True,True
33,NTPS,National Teacher and Principal Survey,1348,True,National Teacher and Principal Survey|NTPS,22,True,national teacher and principal survey,national teacher principal survey,teacher and principal,3,teacher and principal,teacher and principal,ntps,True,True
70,NIAGADS,National Institute on Aging Genetics of Alzhei...,1058,True,National Institute on Aging Genetics of Alzhei...,8,True,national institute on aging genetics of alzhei...,no_match,institute on aging genetics alzheimer's diseas...,7,no_match,no_match,niagads,True,False
130,ECLS,Early Childhood Longitudinal Study-Birth,33,True,Early Childhood Longitudinal Study-Birth|ECLS,4,True,early childhood longitudinal study birth,early childhood longitudinal study,early childhood study-birth,3,no_match,no_match,ecls,True,True


In [32]:
df_label_acronym_ver2[df_label_acronym_ver2['dataset']=='national education longitudinal study']

,acronym,base,text_index,keyword,base_acronym,count_BA,ver2,cleaned_label,dataset,label-keywords,len_label-keywords,match_train-keywords,match_ver1-keywords,acronym_clean,match_train_acronym,match_acronym_ver1
2,NELS,National Education Longitudinal Study,0,True,National Education Longitudinal Study|NELS,258,True,national education longitudinal study,national education longitudinal study,,0,no_match,no_match,nels,True,True
48,NLS,National Longitudinal Study,5,True,National Longitudinal Study|NLS,12,True,national longitudinal study,national education longitudinal study,,0,no_match,no_match,nls,False,True
53,NLTS,National Longitudinal Transition Study,2,True,National Longitudinal Transition Study|NLTS,11,True,national longitudinal transition study,national education longitudinal study,transition,1,no_match,no_match,nlts,False,False
67,CELS,Citizenship Education Longitudinal Study,7493,True,Citizenship Education Longitudinal Study|CELS,9,True,citizenship education longitudinal study,national education longitudinal study,citizenship,1,no_match,citizenship,cels,False,True
146,SEELS,Special Education Elementary Longitudinal Study,2868,True,Special Education Elementary Longitudinal Stud...,3,True,special education elementary longitudinal study,national education longitudinal study,special elementary,2,no_match,no_match,seels,False,False
156,NEILS,National Early Intervention Longitudinal Study,2367,True,National Early Intervention Longitudinal Study...,3,True,national early intervention longitudinal study,national education longitudinal study,early intervention,2,no_match,no_match,neils,False,False
174,ELS,"Education Longitudinal Study,",6,True,"Education Longitudinal Study,|ELS",3,True,education longitudinal study,national education longitudinal study,"study,",1,no_match,no_match,els,True,True
196,SNELS,Special Needs Education Longitudinal Study,7466,True,Special Needs Education Longitudinal Study|SNELS,2,True,special needs education longitudinal study,national education longitudinal study,special needs,2,no_match,no_match,snels,False,False
198,SELS,Seoul Education Longitudinal Study,7502,True,Seoul Education Longitudinal Study|SELS,2,True,seoul education longitudinal study,national education longitudinal study,seoul,1,no_match,no_match,sels,False,False
223,NLSS,National Longitudinal Study of Schools,248,True,National Longitudinal Study of Schools|NLSS,2,True,national longitudinal study of schools,national education longitudinal study,schools,1,no_match,no_match,nlss,False,False


In [33]:
df_tmp = df_label_acronym_ver2[['ver2', 'dataset']]
# df_tmp = ri(df_tmp[df_tmp['dataset'].duplicated()==False])
df_label_acronym_ver1_all['dataset'] = df_label_acronym_ver1_all['base']
df_label_acronym_ver1_all2 = pd.merge(df_label_acronym_ver1_all, df_tmp, on='dataset', how='left')
df_label_acronym_ver1_all2.head()

,acronym,base,base_acronym,train,match,label-keywords,len_label-keywords,match_train-keywords,acronym_clean,match_train_acronym,dataset,ver2
0,NELS,national education longitudinal study,national education longitudinal study|NELS,True,education longitudinal study,,0,no_match,nels,True,national education longitudinal study,True
1,NELS,national education longitudinal study,national education longitudinal study|NELS,True,education longitudinal study,,0,no_match,nels,True,national education longitudinal study,True
2,NELS,national education longitudinal study,national education longitudinal study|NELS,True,education longitudinal study,,0,no_match,nels,True,national education longitudinal study,True
3,NELS,national education longitudinal study,national education longitudinal study|NELS,True,education longitudinal study,,0,no_match,nels,True,national education longitudinal study,True
4,NELS,national education longitudinal study,national education longitudinal study|NELS,True,education longitudinal study,,0,no_match,nels,True,national education longitudinal study,True


In [34]:
print(df_label_acronym_ver1_all2[df_label_acronym_ver1_all2['ver2']!=True].shape)
df_label_acronym_ver1_all2[df_label_acronym_ver1_all2['ver2']!=True].head()

(146, 12)


,acronym,base,base_acronym,train,match,label-keywords,len_label-keywords,match_train-keywords,acronym_clean,match_train_acronym,dataset,ver2
33,HSB,high school and beyond longitudinal dataset,high school and beyond longitudinal dataset|HSB,True,no_match,high school and beyond dataset,5,no_match,hsb,False,high school and beyond longitudinal dataset,NaN
34,NELS,national education longitudinal study of 1988,national education longitudinal study of 1988|...,True,education longitudinal study,1988,1,no_match,nels,True,national education longitudinal study of 1988,NaN
47,NELS,national education longitudinal study 1988 stu...,national education longitudinal study 1988 stu...,True,education longitudinal study,1988 1994,2,no_match,nels,True,national education longitudinal study 1988 stu...,NaN
48,BPS,beginning postsecondary students longitudinal ...,beginning postsecondary students longitudinal ...,True,beginning postsecondary students,beginning postsecondary students,3,beginning postsecondary students,bps,True,beginning postsecondary students longitudinal ...,NaN
54,NLS,national longitudinal survey,national longitudinal survey|NLS,True,no_match,,0,no_match,nls,False,national longitudinal survey,NaN


In [35]:
df_label_acronym_ver2[df_label_acronym_ver2['dataset']=='no_match'].head()

,acronym,base,text_index,keyword,base_acronym,count_BA,ver2,cleaned_label,dataset,label-keywords,len_label-keywords,match_train-keywords,match_ver1-keywords,acronym_clean,match_train_acronym,match_acronym_ver1
17,NSLDS,National Student Loan Data System,233,True,National Student Loan Data System|NSLDS,41,True,national student loan data system,no_match,student loan system,3,no_match,no_match,nslds,False,False
23,NED,National Elevation Dataset,305,True,National Elevation Dataset|NED,30,True,national elevation dataset,no_match,elevation dataset,2,no_match,no_match,ned,False,False
27,ADCS,Alzheimer's Disease Cooperative Study,1070,True,Alzheimer's Disease Cooperative Study|ADCS,27,True,alzheimer s disease cooperative study,no_match,alzheimer's disease cooperative,3,no_match,no_match,adcs,False,False
28,SIRD,Survey of Industrial Research and Development,8253,True,Survey of Industrial Research and Development|...,25,True,survey of industrial research and development,no_match,industrial research and development,4,industrial research and development,no_match,sird,False,False
37,FRSS,Fast Response Survey System,9140,True,Fast Response Survey System|FRSS,20,True,fast response survey system,no_match,fast response system,3,no_match,no_match,frss,False,False


In [36]:
# ver1もBAで処理を
idx_tmp = df_label_acronym_ver2['count_BA']>=2
idx_tmp = idx_tmp & (df_label_acronym_ver2['len_label-keywords']>0)
# idx_tmp = idx_tmp & (df_label_acronym_ver2['match_train-keywords']=='no_match')
# idx_tmp = idx_tmp & (df_label_acronym_ver2['match_ver1-keywords']=='no_match')
# idx_tmp = idx_tmp & (df_label_acronym_ver2['match_train_acronym']==False)
# idx_tmp = idx_tmp & (df_label_acronym_ver2['match_acronym_ver1']==False)
print(df_label_acronym_ver2.shape)
print(df_label_acronym_ver2[idx_tmp].shape)
df_label_acronym_ver2['label'] = df_label_acronym_ver2['base']
df_label_acronym_ver2_filtered = df_label_acronym_ver2[['label', 'cleaned_label', 'acronym', 'acronym_clean']][idx_tmp]
df_label_acronym_ver2_filtered['ver'] = 2
df_label_acronym_ver2_filtered.head(30)

(1172, 16)
(305, 16)


,label,cleaned_label,acronym,acronym_clean,ver
0,Agricultural Resource Management Survey,agricultural resource management survey,ARMS,arms,2
1,Baltimore Longitudinal Study of Aging,baltimore longitudinal study of aging,BLSA,blsa,2
3,Common Core of Data,common core of data,CCD,ccd,2
4,Integrated Postsecondary Education Data System,integrated postsecondary education data system,IPEDS,ipeds,2
5,Survey of Doctorate Recipients,survey of doctorate recipients,SDR,sdr,2
6,Survey of Earned Doctorates,survey of earned doctorates,SED,sed,2
7,Intergovernmental Panel on Climate Change,intergovernmental panel on climate change,IPCC,ipcc,2
8,National Postsecondary Student Aid Study,national postsecondary student aid study,NPSAS,npsas,2
9,Progress in International Reading Literacy Study,progress in international reading literacy study,PIRLS,pirls,2
10,Current Population Survey,current population survey,CPS,cps,2


In [37]:
df_labels = pd.concat([df_label_train2, df_label_acronym_ver1_filtered, df_label_acronym_ver2_filtered])
print(df_labels.shape)
df_labels.to_csv(params['OUTPUT_DIR']+"/df_labels_210619_01.csv", index=None)
df_labels

(887, 5)


,label,cleaned_label,acronym,acronym_clean,ver
0,ADNI,adni,adni,adni,0
1,adni,adni,adni,adni,0
2,Alzheimer's Disease Neuroimaging Initiative (A...,alzheimer s disease neuroimaging initiative adni,adni,adni,0
3,alzheimer s disease neuroimaging initiative adni,alzheimer s disease neuroimaging initiative adni,adni,adni,0
4,Trends in International Mathematics and Scienc...,trends in international mathematics and scienc...,timss,timss,0
...,...,...,...,...,...
302,Beaver Dam Eye Study,beaver dam eye study,BDES,bdes,2
303,Barrow Area Information Database,barrow area information database,BAID,baid,2
304,Alzheimer's Disease Big Data,alzheimer s disease big data,ADBD,adbd,2
305,Agricultural Market Access Database,agricultural market access database,AMAD,amad,2
